In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
crowd_path = base_dir + "crowd/"

Mounted at /content/drive/


In [2]:
import json

with open(crowd_path + "train_m_tree.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

In [3]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  19980 elementi


In [4]:
examples_iter = iter(examples_train)

In [5]:
open(path_corpus_data + 'types_counters_ontonotes.json', 'w').close()

In [6]:
from tqdm import tqdm
from gensim.parsing.preprocessing import remove_stopwords
fp = open(path_corpus_data + 'corpus_data_entire_choi.txt', "w+")  # Open a file
# do file operations.
restricted_set = {}
for i in tqdm(examples_iter):
  if (len(i['y_str'])>1):
    for k in range(0,len(i['y_str'])):
      phrase = []
      phrase = i['left_context_token'] + i['y_str'][k].upper().split() + i['right_context_token']
      fp.write(remove_stopwords(' '.join(phrase)) + '\n')
      if (i['y_str'][k].upper() not in restricted_set):
          restricted_set[i['y_str'][k].upper()]=1
      else:
          restricted_set[i['y_str'][k].upper()]+=1
  else:
    phrase = []
    phrase = i['left_context_token'] + i['y_str'][0].upper().split() + i['right_context_token']
    fp.write(remove_stopwords(' '.join(phrase)) + '\n')
    if (i['y_str'][0].upper() not in restricted_set):
      restricted_set[i['y_str'][0].upper()]=1
    else:
      restricted_set[i['y_str'][0].upper()]+=1

fp.close()


19980it [00:01, 16610.06it/s]


In [7]:
  with open(path_corpus_data + 'types_counters_ontonotes.json', 'a') as fp_3:
      json.dump(restricted_set, fp_3)